In [ ]:
#NN
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(32, input_dim=26, activation = 'elu'),
    keras.layers.Dense(16, activation = 'elu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(8, activation = 'elu'),
    keras.layers.Dense(4, activation = 'elu'),
    keras.layers.Dense(3, activation = 'softmax')
     
    
])